In [2]:
import torch
import random
import numpy as np
from torchvision import transforms

def set_random_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)


set_random_seed(42)

In [3]:
import pandas as pd
dataset = pd.read_pickle('/home/jupyter/datasphere/project/SimpleAllDaatset.pkl')

In [4]:
dataset

,EPP,Iw_i,Iw_f,Phase_i,freq,B_int,beam_dia
0,0.0007,"[3.78723391477998e-21, 1.3045476591790103e-20,...","[6.534839463599934, 8.125660396136668, 10.0987...","[-1.9700863597577265, -1.9707528071286626, -1....","[276.0, 276.2, 276.4, 276.59999999999997, 276....",[1.7583869313023996],1.571429
1,0.0007,"[9.121441553481542e-21, 3.8515583565614827e-20...","[1.9601245885403418, 2.649452397050072, 3.5658...","[-3.125758389086947, -3.170803145611498, -3.23...","[276.0, 276.2, 276.4, 276.59999999999997, 276....",[2.078811529511563],1.571429
2,0.0007,"[1.0639483729707589e-20, 6.246334060439965e-20...","[6.974771788209512, 8.31061092315222, 9.598656...","[-5.595473654476954, -5.503802041395019, -5.33...","[276.0, 276.2, 276.4, 276.59999999999997, 276....",[1.6659182425413812],1.571429
3,0.0007,"[4.8580801352794325e-21, 2.354946902824704e-20...","[359.087409581797, 436.7728652321895, 530.5175...","[3.2477083188578857, 3.029451198598601, 2.8303...","[276.0, 276.2, 276.4, 276.59999999999997, 276....",[2.7793139470289443],1.571429
4,0.0007,"[3.7199019431288444e-21, 1.3186222236151669e-2...","[146.6430985818692, 177.26155414053767, 213.93...","[1.1595752773062131, 1.1712154766759997, 1.294...","[276.0, 276.2, 276.4, 276.59999999999997, 276....",[2.5615524150109077],1.571429
...,...,...,...,...,...,...,...
54995,0.0008,"[1.2983912530621674e-26, 1.50930407485772e-26,...","[8.475212180598568e-09, 1.558093547008606e-08,...","[-2.2570849896698677, 0.5711270257974985, -1.7...","[276.0, 276.2, 276.4, 276.59999999999997, 276....",[2.164080167612508],1.222222
54996,0.0008,"[6.818802581686414e-27, 1.3225236444224792e-26...","[0.00025172922467863167, 0.0004001064309410044...","[15.557303453507055, 14.031108638928915, 13.04...","[276.0, 276.2, 276.4, 276.59999999999997, 276....",[2.7647755794641444],1.222222
54997,0.0008,"[2.6555890591186976e-26, 3.2491945483577303e-2...","[0.00010265014537219348, 0.0001655746476213615...","[7.613429511310831, 8.337759712734929, 9.95259...","[276.0, 276.2, 276.4, 276.59999999999997, 276....",[2.7800095399330895],1.222222
54998,0.0008,"[2.4132537635448508e-26, 1.4199646186345395e-2...","[3.322760812950438e-06, 5.599020364188554e-06,...","[-3.373659703401586, -4.821200853326159, -5.73...","[276.0, 276.2, 276.4, 276.59999999999997, 276....",[2.6242474869085077],1.222222


In [5]:
from sklearn.utils import shuffle

dataset_shuffled = shuffle(dataset, random_state = 42)

In [6]:
dataset_shuffled_norm = []
col_names = ['Iw_norm','Phase']
dataset_shuffled_norm = pd.DataFrame(columns=col_names)

for i in range(len(dataset_shuffled)):#len(dataset_shuffled)
    
    Iw_i = np.log(np.abs(dataset_shuffled.iloc[i]['Iw_i']/dataset_shuffled.iloc[i]['Iw_i'].max()))
    Iw_f = np.log(np.abs(dataset_shuffled.iloc[i]['Iw_f']/dataset_shuffled.iloc[i]['Iw_f'].max()))
    phase = dataset_shuffled.iloc[i]['Phase_i']
    
    d = [np.concatenate((Iw_i, Iw_f)), phase]
    dataset_shuffled_norm.loc[i] = d




In [7]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(torch.tensor(dataset_shuffled_norm['Iw_norm'], dtype=torch.float32), torch.tensor(dataset_shuffled_norm['Phase'], dtype=torch.float32), train_size=0.9, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.8, random_state=42)

/tmp/ipykernel_5169/814915838.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  x_train, x_test, y_train, y_test = train_test_split(torch.tensor(dataset_shuffled_norm['Iw_norm'], dtype=torch.float32), torch.tensor(dataset_shuffled_norm['Phase'], dtype=torch.float32), train_size=0.9, random_state=42)


In [8]:
x_test.shape

torch.Size([5500, 300])

In [9]:
mean = torch.mean(x_train).item()
std = torch.std(x_train).item()

x_train_scaled = (x_train - mean)/std   
x_val_scaled = (x_val - mean)/std 
x_test_scaled = (x_test - mean)/std   
    
trainset = torch.utils.data.TensorDataset(x_train_scaled, y_train)
valset = torch.utils.data.TensorDataset(x_val_scaled, y_val)
testset = torch.utils.data.TensorDataset(x_test_scaled, y_test)

train_loader = torch.utils.data.DataLoader(trainset, batch_size=256, shuffle=True,num_workers=3)
val_loader = torch.utils.data.DataLoader(valset, batch_size=256, shuffle=False,num_workers=3)
test_loader = torch.utils.data.DataLoader(testset, batch_size=256, shuffle=False)

In [10]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [24]:
from torch import nn

class SimpleModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.core = nn.Sequential(
            #nn.Flatten(),
            nn.Linear(300, 500),
            nn.BatchNorm1d(500),
            nn.ReLU(),
            nn.Linear(500, 500),
            nn.BatchNorm1d(500),
            nn.Linear(500, 500),
            nn.BatchNorm1d(500),
            nn.ReLU(),
            nn.Linear(500, 150)
        )

    def forward(self, x):
        return self.core(x)
    
SimpleModel()

SimpleModel(
  (core): Sequential(
    (0): Linear(in_features=300, out_features=500, bias=True)
    (1): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=500, out_features=500, bias=True)
    (4): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): Linear(in_features=500, out_features=500, bias=True)
    (6): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU()
    (8): Linear(in_features=500, out_features=150, bias=True)
  )
)

In [12]:
%pip install -q lightning 


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [13]:
%load_ext tensorboard

In [25]:
import lightning as L

from torch import nn
from torchmetrics import MetricCollection, MeanAbsoluteError,  MeanSquaredError

class LModel(L.LightningModule):
    def __init__(self, model, lr=0.0015, gamma=0.9):
        super().__init__()
        self.save_hyperparameters(logger=False)

        # for optimizer and shaduler
        self.lr = lr
        self.gamma = gamma

        # model
        self.model = model
        self.criterion = MeanSquaredError()

        # metrics
        self.metrics = MetricCollection([
            MeanSquaredError()
        ])
        self.train_metrics = self.metrics.clone(postfix='/train')
        self.val_metrics = self.metrics.clone(postfix='/val')

    def configure_optimizers(self):
        # set optimizer
        optimizer = torch.optim.AdamW(
            self.model.parameters(),
            lr=self.lr,
        )
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "interval": "epoch",  # or 'step'
                "monitor": "loss" # only for self.log
            },
        }

    def training_step(self, batch, batch_idx):
        x, y = batch
        #print('x size:', x.shape)
        #print('y size:', y.shape)
        out = self.model(x)
        #print('out size:', out.shape)
        #filter = torch.where(out > 1e-2, 1.0, 0.0)
        #torch.exp(mean+std*x_train_scaled[20][150:])
        loss = self.criterion(out, y)
        self.train_metrics.update(out, y)
        self.log("loss", loss, prog_bar=True)
        self.log("loss/train", loss, prog_bar=False)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        out = self.model(x)
        loss = self.criterion(out, y)
        self.val_metrics.update(out, y)
        self.log("loss/val", loss, prog_bar=False)

    def on_train_epoch_end(self):
        self.log_dict(self.train_metrics.compute())
        self.train_metrics.reset()

        val_metrics = self.val_metrics.compute()
        self.log_dict(val_metrics)
        self.val_metrics.reset()

    def test_step(self, batch, batch_idx):
        x, y = batch
        out = self.model(x)
        self.metrics.update(out, y)

    def on_test_epoch_end(self):
        self.log_dict(self.metrics.compute())
        self.metrics.reset()

In [26]:
from lightning.pytorch.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    # dirpath='best_ckp/'
    save_last=True,
    every_n_epochs=1,
    save_top_k=1,
    save_weights_only=True,
    monitor="MeanSquaredError/val",
    filename="model",
    mode="min",
)

L.seed_everything(42)

model = SimpleModel()
pl_model = LModel(model)
trainer = L.Trainer(
    max_epochs=65,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0,
    log_every_n_steps=155,
    logger=L.pytorch.loggers.TensorBoardLogger(save_dir="./new_log/"),
)

trainer.fit(
    model=pl_model,
    train_dataloaders=train_loader,
    val_dataloaders=val_loader
)

Seed set to 42
/home/jupyter/.local/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params | Mode 
-----------------------------------------------------------
0 | model         | SimpleModel      | 729 K  | train
1 | criterion     | MeanSquaredError | 0      | train
2 | metrics       | MetricCollection | 0      | train
3 | train_metrics | MetricCollection | 0      | train
4 | val_metrics   | MetricCollection | 0      | train
-----------------------------------------------------------
729 K     Trainable params
0         Non-trainable params
729 K     Total params
2.919     Total estimated mode

Epoch 0: 100%|██████████| 155/155 [00:01<00:00, 119.67it/s, v_num=1, loss=25.60]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 1: 100%|██████████| 155/155 [00:01<00:00, 108.69it/s, v_num=1, loss=22.10]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 2: 100%|██████████| 155/155 [00:01<00:00, 108.31it/s, v_num=1, loss=19.70]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 3: 100%|██████████| 155/155 [00:01<00:00, 111.80it/s, v_num=1, loss=21.90]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 4: 100%|██████████| 155/155 [00:01<00:00, 108.97it/s, v_num=1, loss=21.00]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 5: 100%|██████████| 155/155 [00:01<00:00, 110.83it/s, v_num=1, loss=21.50]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 6: 100%|██████████| 155/155 [00:01<00:00, 112.90it/s, v_num=1, loss=23.30]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 7: 100%|██████████| 155/155 [00:01<00:00, 112.89it/s, v_num=1, loss=20.20]
Validation: |          | 0/? [

`Trainer.fit` stopped: `max_epochs=65` reached.


Epoch 64: 100%|██████████| 155/155 [00:01<00:00, 85.01it/s, v_num=1, loss=20.20]


In [27]:
trainer.test(model=pl_model, dataloaders=test_loader, ckpt_path="best")

Restoring states from the checkpoint path at ./new_log/lightning_logs/version_1/checkpoints/model.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at ./new_log/lightning_logs/version_1/checkpoints/model.ckpt
/home/jupyter/.local/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 22/22 [00:00<00:00, 291.23it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│     MeanSquaredError      │    20.430410385131836     │
└───────────────────────────┴───────────────────────────┘

[{'MeanSquaredError': 20.430410385131836}]